# 1. Import Required Libraries
Import libraries such as pandas, numpy, and transformers for data handling and model training.

In [1]:
import sys
import os

# Add the parent directory to the path so the package is importable
sys.path.append(os.path.abspath(".."))

from llm_data_quality_assistant import pipeline
from llm_data_quality_assistant.corruptor import RowCorruptionTypes, CellCorruptionTypes
from llm_data_quality_assistant.enums import Models
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv
import numpy as np

load_dotenv()

True

# 2. Load and Explore EudraCT Data
Load the EudraCT dataset and perform exploratory data analysis to understand its structure and content.

In [2]:
corrupt_dataset = pd.read_csv("../datasets/parker_datasets/eudract/eudract_corrupted_first1000.csv"
)
# Drop the columns that contain wrong values accoording to Bronsealer & Acosta (2023)
try:
    corrupt_dataset.drop(columns=["placebo", "active_comparator"], inplace=True)
except KeyError:
    print("Columns 'placebo' and 'active_comparator' not found in the dataset.")

gold_standard = pd.read_csv(
    "../datasets/parker_datasets/eudract/eudract_cleaned_gold_first1000.csv"
)

with open("../datasets/parker_datasets/eudract/eudract.partialkey", "r") as f:
    partial_keys = f.read()

with open("../datasets/parker_datasets/eudract/eudract.rules", "r") as f:
    rules = f.read()

print(partial_keys)
print(rules)
print(corrupt_dataset.head(2))
print(gold_standard.head(2))
print(type(gold_standard.get("eudract_number").iloc[0]))
print(type(corrupt_dataset.get("eudract_number").iloc[0]))

Columns 'placebo' and 'active_comparator' not found in the dataset.
eudract_number ->
    single_blind,
    double_blind,
    open,
    controlled,
    placebo,
    active_comparator,
    randomised,
    crossover,
    parallel_group,
    arms

-- Attributes
@open:STRING
@single_blind:STRING
@double_blind:STRING
@randomised:STRING
@controlled:STRING
@placebo:STRING
@active_comparator:STRING
@crossover:STRING
@parallel_group:STRING
@arms:STRING

-- Overview of attributes
open notin {'Yes', 'No'}
single_blind notin {'Yes', 'No'}
double_blind notin {'Yes', 'No'}
randomised notin {'Yes', 'No'}
controlled notin {'Yes', 'No'}
placebo notin {'Yes', 'No'}
active_comparator notin {'Yes', 'No'}
crossover notin {'Yes', 'No'}
parallel_group notin {'Yes', 'No'}
arms notin {'0', '1', '2+'}

-- eudract rules for masking
open == 'Yes' & single_blind == 'Yes'
open == 'Yes' & double_blind == 'Yes'
single_blind == 'Yes' & double_blind == 'Yes'
open == 'No' & single_blind == 'No' & double_blind == 'No'

-

# 3. Clean and Merge Data with LLM
Use the LLM pipeline to clean and merge the corrupted dataset using the provided rules and evaluate the results.

In [3]:
from llm_data_quality_assistant.pipeline import Pipeline
from llm_data_quality_assistant.enums import Models

eudract_primary_key = "eudract_number"
model = Models.GeminiModels.GEMINI_2_0_FLASH_LITE
rpm = 30

merged_df = Pipeline.merge_with_llm(
    dataset=corrupt_dataset,
    primary_key=eudract_primary_key,
    model_name=model,
    rpm=rpm,
    additional_prompt=rules,
    verbose=True
)

print(merged_df)
print(gold_standard)
print(corrupt_dataset)

[
  {
    "eudract_number": "2004-00
  {
    "eudract_number": "2004-000232-91",
    "arms": "2+",0232-91",
    "arms": "2+",
    "controlled": "Yes",
    "crossover": "No",
    "double_blind": "No",
    "open":
    "controlled": "Yes",
    "crossover": "No",
    "double_blind": "No",
    "open": "Yes",
    "parallel_group": "Yes",
    "randomised": "Yes",
    "single_blind": "No" "Yes",
    "parallel_group": "Yes",
    "randomised": "Yes",
    "single_blind": "No"
  }
]
  }
][[
  {
  {
    "eudract_number": "2004-00
    "eudract_number": "2004-000299-15",
    "arms": "2+",0299-15",
    "arms": "2+",
    "controlled": "Yes",
    "crossover": "No",
    "double_blind": "Yes",
    "open":
    "controlled": "Yes",
    "crossover": "No",
    "double_blind": "Yes",
    "open": "No",
    "parallel_group": "Yes",
    "randomised": "Yes",
    "single_blind": "No" "No",
    "parallel_group": "Yes",
    "randomised": "Yes",
    "single_blind": "No"
  }
]
  }
][[
  {
  {
    "eudract_number": "200

# 4. Evaluate the Results
Evaluate the cleaned dataset using micro and macro evaluation metrics.

In [4]:
stats_micro = Pipeline.evaluate_micro(
    gold_standard=gold_standard,
    cleaned_dataset=merged_df,
    corrupted_dataset=corrupt_dataset
)
print("====================================")
print("MICRO EVALUATION RESULTS")
print("====================================")
pprint(stats_micro)

stats_macro = Pipeline.evaluate_macro(
    gold_standard=gold_standard,
    cleaned_dataset=merged_df,
    corrupted_dataset=corrupt_dataset
)
print("====================================")
print("MACRO EVALUATION RESULTS")
print("====================================")
pprint(stats_macro)

MICRO EVALUATION RESULTS
{'accuracy': 0.9443912472958116,
 'column_names': ['eudract_number',
                  'arms',
                  'controlled',
                  'crossover',
                  'double_blind',
                  'open',
                  'parallel_group',
                  'randomised',
                  'single_blind'],
 'f1_score': 0.6592785745328118,
 'false_negative': 1445,
 'false_negative_rate': 0.48784604996623904,
 'false_positive': 123,
 'false_positive_rate': 0.0048741826827818505,
 'num_columns': 9,
 'num_rows': 3133,
 'precision': 0.925,
 'recall': 0.512153950033761,
 'true_negative': 25112,
 'true_positive': 1517}
MACRO EVALUATION RESULTS
{'column_names': ['eudract_number',
                  'arms',
                  'controlled',
                  'crossover',
                  'double_blind',
                  'open',
                  'parallel_group',
                  'randomised',
                  'single_blind'],
 'num_columns': 9,
 'num_rows